In [16]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

/Applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [10]:
# import our chat-bot intents file
import json
intents = json.load(open('/Users/markwoolnough/scrapy/chatbot/intents.json'))

In [39]:
words = []
classes = []
documents = []
ignore_words = ['?',',','-']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

26 documents
14 classes ['AccountInfo', 'Balance', 'DigitalBanking', 'GoingAbroad', 'Iban', 'LostStolen', 'NoCardReader', 'Overdraft', 'PaymentDuration', 'goodbye', 'greeting', 'hours', 'opentoday', 'thanks']
80 unique stemmed words ["'s", 'a', 'abroad', 'account', 'am', 'and', 'anyon', 'ar', 'avail', 'bal', 'bank', 'be', 'between', 'bic', 'block', 'bye', 'can', 'card', 'card-reader', 'cod', 'dam', 'day', 'debit', 'diff', 'digit', 'do', 'find', 'for', 'going', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'ib', 'in', 'is', 'lat', 'limit', 'long', 'lost', 'mak', 'my', "n't", 'numb', 'op', 'or', 'ord', 'out', 'overdraft', 'pay', 'quickest', 'reg', 'replac', 'report', 'sav', 'see', 'should', 'sort', 'stol', 'tak', 'thank', 'that', 'the', 'ther', 'to', 'today', 'transf', 'want', 'way', 'what', 'when', 'wher', 'wil', 'yo', 'you']


In [40]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [41]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.02515 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.02515 - acc: 0.9996 -- iter: 24/26
Training Step: 4000  | total loss: 0.02470 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.02470 - acc: 0.9996 -- iter: 26/26
--
INFO:tensorflow:/Users/markwoolnough/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [42]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )